In [ ]:
import csv
import re
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from wordcloud import WordCloud

0.将QQ导入的聊天记录的txt格式转换为csv格式

In [ ]:
# 先把名字修改了
TXT = '群名'
myTXT = TXT + '.txt'
myCSV = TXT + '.csv'

In [ ]:
# 转换格式
with open(myTXT, 'r', encoding='utf-8') as f:
    lines = f.readlines()

with open(myCSV, 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['时间', '用户', '聊天'])
    for i in range(0, len(lines), 3):
        match = re.search(r'(202\d-\d{2}-\d{2} \d{2}:\d{2}:\d{2}) .*?(\(\d+\)|\<.*\>)', lines[i])
        if match:
            timestamp = match.group(1)
            username = match.group(2)
            chat = lines[i+1].strip()
            writer.writerow([timestamp, username, chat])


In [ ]:
# 读取csv
data = pd.read_csv(myCSV, parse_dates=['时间'])

In [ ]:
np.sum(data.isnull(),axis=0)

In [ ]:
data.astype({'时间':'datetime64','用户':'str','聊天':'str'})
data['date'] = data.时间.dt.date
data['hour'] = data.时间.dt.hour

In [ ]:
# 记录一共多少用户
data.用户.nunique()

In [ ]:
df = data.groupby('date').用户.nunique().reset_index()
plt.figure(figsize=(16,5))
sns.set(style='darkgrid',context='notebook',font_scale=1.2)
sns.lineplot(data=df,x='date',y='用户') 
plt.title('群聊每日发言用户数量',pad=15,fontdict={'fontsize':20})
plt.xticks(df.date,rotation=70)
mpl.rcParams['font.sans-serif'] = ['SimHei']

In [ ]:
df = data.groupby('date').聊天.count().reset_index()

plt.figure(figsize=(16,5))
sns.set(style='darkgrid',context='notebook',font_scale=1.2)
sns.lineplot(data=df,x='date',y='聊天') 
plt.title('群聊每日消息数',pad=15,fontdict={'fontsize':20})
plt.xticks(df.date,rotation=70)
mpl.rcParams['font.sans-serif'] = ['SimHei']

In [ ]:
df = data.groupby('hour').聊天.count().reset_index()
df['hour'] = df.hour.astype('int64')
df.sort_values('hour',inplace=True)

plt.figure(figsize=(16,5))
sns.set(style='darkgrid',context='notebook',font_scale=1.2)
sns.lineplot(data=df,x='hour',y='聊天') 
plt.title('群聊每时段消息数',pad=15,fontdict={'fontsize':20})
plt.xticks(df.hour,rotation=70)
mpl.rcParams['font.sans-serif'] = ['SimHei']

In [ ]:
df = data.groupby('用户').size().sort_values(ascending=False)
df.head(10)

In [ ]:
# 选择要绘制的用户
user = '(QQ号)'
df = data.groupby(['date', '用户']).size().loc[:, user]

df.plot(kind='line')
plt.title(f'{user} 的每日消息数')
plt.xlabel('date')
plt.ylabel('消息数')
plt.xticks(rotation=70)
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.show()